## Introducing PaperClip: A Personal Learning Assistant that learns and answers questions about scientific paper of interest from arxiv, in a readable, short and digestable format. 
I am building it using LangChain and OpenAI model (gpt-3.5-turbo)

#### Inputs: 
- Input a arxiv paper ID as input (ex: 1706.03762)
- user question (input)


##### Output:
smart answer - a simple summary of scientific paper of interest

##### Concepts I am demostrating are below:
- Role prompting to mimic learning assistant role 
- Vector database to store the data source and support semantic search to retrieve relevant context
- Personalized response 



#### Evaluation:



#### References/Credit:
https://arxiv.org/pdf/2309.15217.pdf
https://medium.com/@akash.hiremath25/unlocking-the-power-of-intelligence-building-an-application-with-gemini-python-and-faiss-for-eb9a055d2429



In [18]:
%%capture
# update or install the necessary libraries
!pip install --upgrade openai
!pip install -U sentence-transformers
!pip install loader
!pip install lxml
!pip install arxiv
!pip install PyPDF2
!pip install tiktoken
!pip install langchain
!pip install sentence-transformers
!pip install numpy



In [19]:
import openai
import os
import pandas as pd
from langchain.llms import OpenAI
import IPython
from sentence_transformers import SentenceTransformer
import numpy as np

# API configuration
openai.api_key = os.getenv("OPENAI_API_KEY")

In [20]:
import re
from langchain.document_loaders.parsers import BS4HTMLParser, PDFMinerParser
from langchain.document_loaders.parsers.generic import MimeTypeBasedParser
from langchain.document_loaders.parsers.txt import TextParser
from langchain_community.document_loaders import Blob
import requests
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import BSHTMLLoader
import arxiv
import PyPDF2
import io
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.llms import OpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain import hub
from langchain import PromptTemplate, LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")

# Function to convert PDF to text
def pdf_to_txt(input_file):
    with open(input_file, 'rb') as f:
        pdf_reader = PyPDF2.PdfReader(f)
        text = ''

        # Extract text from each page of the PDF
        for page_num in range(len(pdf_reader.pages)):
            text += pdf_reader.pages[page_num].extract_text()
    
    return text

#  Function to prepare the index for querying
# 
def prepare_index(output_file):
     loader = TextLoader(output_file)
     index = VectorstoreIndexCreator().from_loaders([loader])
     return index
# 

### Here are step by step process I followed to build PaperClip

- Step 1: User is askied to provide Arxiv ID of scientific paper of his/her interest. I am using arxiv to the download that paper.
- Step 2: Convert the paper pdf version into text for better loading and parsing
- Step 3: Creating CharacterTextSplitter to split the paper content into chunks of size 3000 tokens with overlap of 100 between chunks
- Step 4: Creating embeddings of the chunks and storing them in vector databse (ChromaDB)
- Step 5: Using similarity search (cosine similarity) to retrive relevant chunks of context
- Step 6: Using langchain's load_qa_with_sources_chain to generate response based on the retrived information. Returning output answer only. Here I'm using several prompt engineering techniques to retrive answer differently and refine it to be more closer to what user might expect.

In [21]:

#input_file = '/Users/apoorvaacharya/Documents/GitHub/maven-pe-for-llms-8/PaperClip/PaperLLM/PaperDB/2303.18223.pdf'
#output_file = '/Users/apoorvaacharya/Documents/GitHub/maven-pe-for-llms-8/PaperClip/PaperLLM/PaperDB/2303.18223.txt'

##Ask for Arxiv ID and prepare the index
arxiv_id = input("Enter Arxiv ID (e.g. 2303.17580): ")
search = arxiv.Search(id_list=[arxiv_id])
paper = next(search.results())
paper.download_pdf(filename="downloaded-paper.pdf")

input_file = 'downloaded-paper.pdf'
output_file = 'converted-paper.txt'

## loading data
text = pdf_to_txt(input_file)

# Save the text to a file
with io.open(output_file, 'w', encoding='utf-8') as f:
    f.write(text)

# Prepare the index for querying
index = prepare_index(output_file)

/var/folders/0m/d5qp0g8x6l18qjc_gydmnq9r0000gp/T/ipykernel_79212/547467742.py:7: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  paper = next(search.results())


In [22]:
# MEthod 1: Splitting the text

#text_splitter = CharacterTextSplitter(chunk_size=3000, chunk_overlap=100, separator="\n")
#text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=100, separator="\n")

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap =  50)
texts = text_splitter.split_text(text)

# embeddings  from OpenAI
embeddings = OpenAIEmbeddings()

## creating vector store
docsearch = Chroma.from_texts(texts, embeddings, metadatas=[{"source": str(i)} for i in range(len(texts))])

## rag retrieval based on similarity search
query = "What is the paper title?"
docs = docsearch.similarity_search_with_relevance_scores(query)


In [23]:


# contexts = list(map(lambda doci: doci[0].page_content, docs))
# scores =  list(map(lambda doci: doci[1], docs))

# context_dict = {"contexts": contexts,"scores": scores}
# #context_dict = sorted(context_dict)
# context_dict

## Contexts retrieved are below 
docs_iter = [item[0] for item in docs]
docs_iter

[Document(page_content='- August 4, Volume 1: Long Papers , 2017, pp. 1601–1611.\n[559] Y. Bisk, R. Zellers, R. L. Bras, J. Gao, and Y. Choi,\n“PIQA: reasoning about physical commonsense in\nnatural language,” in The Thirty-Fourth AAAI Confer-\nence on Artificial Intelligence, AAAI 2020, The Thirty-\nSecond Innovative Applications of Artificial Intelligence\nConference, IAAI 2020, The Tenth AAAI Symposium\non Educational Advances in Artificial Intelligence, EAAI\n2020, New York, NY, USA, February 7-12, 2020 , 2020,', metadata={'source': '1424'}),
 Document(page_content='tational Linguistics (Volume 1: Long Papers), ACL 2022,\nDublin, Ireland, May 22-27, 2022 , S. Muresan, P . Nakov,\nand A. Villavicencio, Eds., 2022, pp. 3470–3487.\n[167] S. H. Bach, V . Sanh, Z. X. Yong, A. Webson, C. Raffel,\nN. V . Nayak, A. Sharma, T. Kim, M. S. Bari, T. F ´evry,\nZ. Alyafeai, M. Dey, A. Santilli, Z. Sun, S. Ben-David,\nC. Xu, G. Chhablani, H. Wang, J. A. Fries, M. S.\nAlShaibani, S. Sharma, U. Tha

In [24]:

#cc = list(map(lambda doci: doci[0].page_content, docs))


## Using Langchain Q&A chain to generate the responses using OpenAI model 

 ##### Step 1. Answer generation without prompting

In [25]:
## modelling response generation without prompting

chain = load_qa_with_sources_chain(OpenAI(temperature=0.3), chain_type="stuff")
chain({"input_documents": docs_iter, "question": query}, return_only_outputs=True)



{'output_text': ' The paper title is "Transformers: State-of-the-art natural language processing."\nSOURCES: 1231'}

In [26]:
# # Main loop for queries
# while True:
#     # Ask for the user's query
#     user_query = input("Enter your query (or type 'exit' to quit): ")
#     print(user_query)
#     if user_query.lower() == 'exit':
#         break

#     # Query the index if it exists
#     if index:
#         response = index.query(user_query)
#         print(f"Response: {response}")


### Improving Answer with Prompting
##### Step 2. Modelling response generation with prompting to return only based on the context retrived 

In [27]:

## modelling response generation with prompting to return only based on the context retrived 

retriever = docsearch.as_retriever()
#prompt = hub.pull("rlm/rag-prompt")

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


template = """You are a personal learning agent. 
Output "NA" if you are not able to answer the question. 
Answer the question based only on the following context: {context}

Question: {question}

"""


prompt = ChatPromptTemplate.from_template(template)

model = OpenAI(temperature=0.3)
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

answer2 = chain.invoke(query)
answer2

#answer_withprompt = output['output_text']

'PIQA: reasoning about physical commonsense in natural language'

##### Step 3. Response generation with prompting  not limited to the context retrived
learning - Its not reliable

In [28]:
template = """
Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). 
If you don't know the answer, just say that you don't know. Don't try to make up an answer. Keep the answer short.
ALWAYS return a "SOURCES" part in your answer. 

=========
{summaries}
=========

Given the summary above, help answer the following question from the user:

Question: {question}
"""

# create a prompt template
PROMPT = PromptTemplate(template=template, input_variables=["summaries", "question"])

# query 
chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="stuff", prompt=PROMPT)
chain({"input_documents": docs_iter, "question": query}, return_only_outputs=True)

{'output_text': '\nAnswer: "Transformers: State-of-the-art natural language processing" (SOURCES: 1231)'}

### Answer without prompt to translate into different language

In [29]:
### language translation prompt
from operator import itemgetter


template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the mentioned language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke({"question": query, "context": itemgetter("question") | retriever, "language": "italian"})

'\nIl titolo del paper è "Transformers: State-of-the-art natural language processing".'

In [30]:
eval_arrow_dataset

NameError: name 'eval_arrow_dataset' is not defined

In [ ]:
# Other method

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_text(text)

# embeddings  from OpenAI
embeddings = OpenAIEmbeddings()

vectorstore = Chroma.from_texts(texts, embeddings, metadatas=[{"source": str(i)} for i in range(len(texts))])


# Retrieve and generate using the relevant snippets of the blog.
retriever = docsearch.as_retriever()
#prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=1)

rag_chain = (
    {"context": retriever | texts, "question": RunnablePassthrough()}
    | llm
    | StrOutputParser()
)

In [ ]:
import json

from langchain_community.document_transformers import DoctranQATransformer
from langchain_core.documents import Document


documents = [Document(page_content=text)]
qa_transformer = DoctranQATransformer()
transformed_document = qa_transformer.transform_documents(documents)


In [ ]:
def get_completion(messages, model="gpt-3.5-turbo-0613", temperature=0, max_tokens=300):
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return response.choices[0].message.content

